In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pandas_datareader.data as web
import time
import tensorflow
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import logging
import math
import os
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from sklearn.metrics import mean_squared_error
from keras.models import load_model

Using TensorFlow backend.


In [2]:
np.random.seed(47)

In [3]:
ohlson_info = pd.read_excel('./data/kospi200_4_ohlson.xlsx')
stock_info = pd.read_excel('./data/10년치_주가_LSTM_비율_올슨.xlsx')

In [4]:
ohlson_info['순자산'] = ohlson_info['자산(*)(요약)(IFRS연결)(백만원)'] - ohlson_info['부채(*)(요약)(IFRS연결)(백만원)']

In [5]:
del ohlson_info['자산(*)(요약)(IFRS연결)(백만원)']
del ohlson_info['부채(*)(요약)(IFRS연결)(백만원)']

In [6]:
ohlson_info.columns

Index(['회사명', '거래소코드', '회계년도', '판매비와 관리비(물류원가 등 포함)(요약)(3개월)(IFRS연결)(백만원)',
       '기타이익(요약)(3개월)(IFRS연결)(백만원)', '기타손실(요약)(3개월)(IFRS연결)(백만원)',
       '법인세비용차감전순이익(손실)(요약)(3개월)(IFRS연결)(백만원)', '법인세비용(요약)(3개월)(IFRS연결)(백만원)',
       '계속영업이익(손실)(요약)(3개월)(IFRS연결)(백만원)', '당기순이익(손실)(요약)(3개월)(IFRS연결)(백만원)',
       '기타포괄손익(요약)(3개월)(IFRS연결)(백만원)', '총포괄손익(요약)(3개월)(IFRS연결)(백만원)',
       '배당금(*)(요약)(IFRS)(백만원)', '순자산'],
      dtype='object')

In [7]:
ohlson_info = ohlson_info[['회사명', '거래소코드', '회계년도','순자산', '판매비와 관리비(물류원가 등 포함)(요약)(3개월)(IFRS연결)(백만원)',
       '기타이익(요약)(3개월)(IFRS연결)(백만원)', '기타손실(요약)(3개월)(IFRS연결)(백만원)',
       '법인세비용차감전순이익(손실)(요약)(3개월)(IFRS연결)(백만원)', '법인세비용(요약)(3개월)(IFRS연결)(백만원)',
       '계속영업이익(손실)(요약)(3개월)(IFRS연결)(백만원)', '당기순이익(손실)(요약)(3개월)(IFRS연결)(백만원)',
       '기타포괄손익(요약)(3개월)(IFRS연결)(백만원)', '총포괄손익(요약)(3개월)(IFRS연결)(백만원)',
       '배당금(*)(요약)(IFRS)(백만원)']]

In [8]:
ohlson_info.columns = ['회사명', '거래소코드', '회계년도', '순자산(백만원)', '판매비와 관리비(물류원가 등 포함)(백만원)',
       '기타이익(백만원)', '기타손실(백만원)', '법인세비용차감전순이익(백만원)', '법인세비용(백만원)',
       '계속영업이익(손실)(백만원)', '당기순이익(백만원)', '기타포괄손익(백만원)', '총포괄손익(백만원)', '배당금(백만원)']

In [9]:
stock_info.columns

Index(['회사명', '거래소코드', '회계연도', '거래년도(*)', '거래월(*)', '거래일수', '최고가(원)', '최저가(원)',
       '산술평균(원)', '종가(원)', '3개월뒤종가(원)', '거래량(주)', '거래대금(원)'],
      dtype='object')

In [10]:
stock_info.columns = ['회사명', '거래소코드', '회계년도', '거래년도', '거래월', '거래일수', '최고가(원)', '최저가(원)',
       '산술평균(원)', '종가(원)', '3개월뒤종가(원)','거래량(주)', '거래대금(원)']

In [11]:
ohlson_data = pd.merge(ohlson_info, stock_info, on = ['회사명','거래소코드','회계년도'],how = 'left')

In [12]:
ohlson_data.isna().sum()

회사명                           0
거래소코드                         0
회계년도                          0
순자산(백만원)                    227
판매비와 관리비(물류원가 등 포함)(백만원)    751
기타이익(백만원)                   751
기타손실(백만원)                   751
법인세비용차감전순이익(백만원)            751
법인세비용(백만원)                  751
계속영업이익(손실)(백만원)             751
당기순이익(백만원)                  751
기타포괄손익(백만원)                 751
총포괄손익(백만원)                  751
배당금(백만원)                      4
거래년도                        152
거래월                         152
거래일수                        152
최고가(원)                      152
최저가(원)                      152
산술평균(원)                     152
종가(원)                       152
3개월뒤종가(원)                   343
거래량(주)                      152
거래대금(원)                     152
dtype: int64

# NA 값 있는 행 삭제

In [13]:
ohlson_data_non_na = ohlson_data[ohlson_data['기타이익(백만원)'].isna() == False]
ohlson_data_non_na = ohlson_data_non_na[ohlson_data_non_na['순자산(백만원)'].isna() == False]
ohlson_data_non_na = ohlson_data_non_na[ohlson_data_non_na['거래년도'].isna() == False]
ohlson_data_non_na = ohlson_data_non_na[ohlson_data_non_na['3개월뒤종가(원)'].isna() == False]

In [14]:
ohlson_data_non_na.isna().sum()

회사명                         0
거래소코드                       0
회계년도                        0
순자산(백만원)                    0
판매비와 관리비(물류원가 등 포함)(백만원)    0
기타이익(백만원)                   0
기타손실(백만원)                   0
법인세비용차감전순이익(백만원)            0
법인세비용(백만원)                  0
계속영업이익(손실)(백만원)             0
당기순이익(백만원)                  0
기타포괄손익(백만원)                 0
총포괄손익(백만원)                  0
배당금(백만원)                    0
거래년도                        0
거래월                         0
거래일수                        0
최고가(원)                      0
최저가(원)                      0
산술평균(원)                     0
종가(원)                       0
3개월뒤종가(원)                   0
거래량(주)                      0
거래대금(원)                     0
dtype: int64

In [15]:
ohlson_data_non_na.head()

,회사명,거래소코드,회계년도,순자산(백만원),판매비와 관리비(물류원가 등 포함)(백만원),기타이익(백만원),기타손실(백만원),법인세비용차감전순이익(백만원),법인세비용(백만원),계속영업이익(손실)(백만원),...,거래년도,거래월,거래일수,최고가(원),최저가(원),산술평균(원),종가(원),3개월뒤종가(원),거래량(주),거래대금(원)
0,(주)BNK금융지주,138930,2011/12,2954948.0,162799.0,2432.0,6144.0,107555.0,28290.0,79264.0,...,2011.0,12.0,21.0,11950.0,10300.0,11126.0,11050.0,13300.0,16665990.0,1.857773e+09
1,(주)BNK금융지주,138930,2012/12,3255216.0,174999.0,740.0,5772.0,68185.0,16119.0,52065.0,...,2012.0,12.0,18.0,13900.0,11900.0,12900.0,13200.0,15300.0,9487437.0,1.226757e+09
2,(주)BNK금융지주,138930,2013/12,3600865.0,178115.0,5704.0,6394.0,26775.0,5354.0,21420.0,...,2013.0,12.0,20.0,16500.0,14950.0,15663.0,16000.0,15050.0,11624471.0,1.819181e+09
3,(주)BNK금융지주,138930,2014/12,5463944.0,323601.0,6054.0,19516.0,500118.0,1912.0,498205.0,...,2014.0,12.0,21.0,15900.0,14250.0,15114.0,14500.0,15200.0,8931317.0,1.340248e+09
4,(주)BNK금융지주,138930,2015/12,6190683.0,337719.0,10084.0,16622.0,37321.0,15367.0,21954.0,...,2015.0,12.0,21.0,9700.0,8180.0,8716.0,8420.0,9700.0,33371878.0,2.902300e+09


In [16]:
ohlson_data_non_na.columns

Index(['회사명', '거래소코드', '회계년도', '순자산(백만원)', '판매비와 관리비(물류원가 등 포함)(백만원)',
       '기타이익(백만원)', '기타손실(백만원)', '법인세비용차감전순이익(백만원)', '법인세비용(백만원)',
       '계속영업이익(손실)(백만원)', '당기순이익(백만원)', '기타포괄손익(백만원)', '총포괄손익(백만원)',
       '배당금(백만원)', '거래년도', '거래월', '거래일수', '최고가(원)', '최저가(원)', '산술평균(원)',
       '종가(원)', '3개월뒤종가(원)', '거래량(주)', '거래대금(원)'],
      dtype='object')

In [17]:
ohlson_data = ohlson_data_non_na

In [18]:
ohlson_data

,회사명,거래소코드,회계년도,순자산(백만원),판매비와 관리비(물류원가 등 포함)(백만원),기타이익(백만원),기타손실(백만원),법인세비용차감전순이익(백만원),법인세비용(백만원),계속영업이익(손실)(백만원),...,거래년도,거래월,거래일수,최고가(원),최저가(원),산술평균(원),종가(원),3개월뒤종가(원),거래량(주),거래대금(원)
0,(주)BNK금융지주,138930,2011/12,2954948.0,162799.0,2432.0,6144.0,107555.0,28290.0,79264.0,...,2011.0,12.0,21.0,11950.0,10300.0,11126.0,11050.0,13300.0,16665990.0,1.857773e+09
1,(주)BNK금융지주,138930,2012/12,3255216.0,174999.0,740.0,5772.0,68185.0,16119.0,52065.0,...,2012.0,12.0,18.0,13900.0,11900.0,12900.0,13200.0,15300.0,9487437.0,1.226757e+09
2,(주)BNK금융지주,138930,2013/12,3600865.0,178115.0,5704.0,6394.0,26775.0,5354.0,21420.0,...,2013.0,12.0,20.0,16500.0,14950.0,15663.0,16000.0,15050.0,11624471.0,1.819181e+09
3,(주)BNK금융지주,138930,2014/12,5463944.0,323601.0,6054.0,19516.0,500118.0,1912.0,498205.0,...,2014.0,12.0,21.0,15900.0,14250.0,15114.0,14500.0,15200.0,8931317.0,1.340248e+09
4,(주)BNK금융지주,138930,2015/12,6190683.0,337719.0,10084.0,16622.0,37321.0,15367.0,21954.0,...,2015.0,12.0,21.0,9700.0,8180.0,8716.0,8420.0,9700.0,33371878.0,2.902300e+09
5,(주)BNK금융지주,138930,2016/12,7087153.0,344820.0,6875.0,26367.0,69494.0,20886.0,48608.0,...,2016.0,12.0,21.0,9240.0,8620.0,8937.0,8680.0,9200.0,16097490.0,1.435937e+09
11,(주)DB하이텍,990,2011/12,308062.0,127164.0,0.0,0.0,-93007.0,4940.0,-97947.0,...,2011.0,12.0,21.0,8000.0,6900.0,7352.0,7250.0,10200.0,7058061.0,5.239409e+08
12,(주)DB하이텍,990,2012/12,290893.0,112405.0,19599.0,13857.0,-48771.0,-1044.0,-47726.0,...,2012.0,12.0,18.0,6660.0,6000.0,6357.0,6330.0,6250.0,1874968.0,1.192467e+08
13,(주)DB하이텍,990,2013/12,204757.0,25042.0,1453.0,5848.0,-41308.0,632.0,-41940.0,...,2013.0,12.0,20.0,7970.0,4955.0,5506.0,7530.0,6700.0,9662005.0,6.343193e+08
14,(주)DB하이텍,990,2014/12,120703.0,22435.0,3053.0,13464.0,-29514.0,-10742.0,-18772.0,...,2014.0,12.0,21.0,5740.0,4335.0,5003.0,4615.0,5990.0,5196476.0,2.608675e+08


In [19]:
ohlson_data.columns

Index(['회사명', '거래소코드', '회계년도', '순자산(백만원)', '판매비와 관리비(물류원가 등 포함)(백만원)',
       '기타이익(백만원)', '기타손실(백만원)', '법인세비용차감전순이익(백만원)', '법인세비용(백만원)',
       '계속영업이익(손실)(백만원)', '당기순이익(백만원)', '기타포괄손익(백만원)', '총포괄손익(백만원)',
       '배당금(백만원)', '거래년도', '거래월', '거래일수', '최고가(원)', '최저가(원)', '산술평균(원)',
       '종가(원)', '3개월뒤종가(원)', '거래량(주)', '거래대금(원)'],
      dtype='object')

In [20]:
ohlson_data = ohlson_data[['회사명', '거래소코드', '회계년도', '순자산(백만원)', '판매비와 관리비(물류원가 등 포함)(백만원)',
       '기타이익(백만원)', '기타손실(백만원)', '법인세비용차감전순이익(백만원)', '법인세비용(백만원)',
       '계속영업이익(손실)(백만원)', '당기순이익(백만원)', '기타포괄손익(백만원)', '총포괄손익(백만원)',
       '배당금(백만원)', '3개월뒤종가(원)']]

# 회사별로 리스트에 넣기

In [21]:
list_code = list(set(ohlson_data['거래소코드']))
list_company = list(set(ohlson_data['회사명']))

In [22]:
list_company1 = []
for i in range(len(list_code)):
    a = ohlson_data[ohlson_data['거래소코드'] == list_code[i]]
    list_company1.append(a)

# 자료 개수가 같은 것만 모으기 (행의 개수: 7 / 11~17년)

In [23]:
list_company2 = []
for i in range(len(list_company1)):
    a = len(list_company1[i])
    list_company2.append(a)
    
# list_company2 - len이 7이 젤 많음

In [24]:
list_company3 = []
for i in range(len(list_company2)):
    if list_company2[i] == 6:
        list_company3.append(list_company1[i])

In [25]:
len(list_company3) # 길이가 7이고 na값이 0인 기업들만 모아둠

128

In [26]:
list_company3[0]

,회사명,거래소코드,회계년도,순자산(백만원),판매비와 관리비(물류원가 등 포함)(백만원),기타이익(백만원),기타손실(백만원),법인세비용차감전순이익(백만원),법인세비용(백만원),계속영업이익(손실)(백만원),당기순이익(백만원),기타포괄손익(백만원),총포괄손익(백만원),배당금(백만원),3개월뒤종가(원)
866,SK이노베이션(주),96770,2011/12,14831610.0,562777.0,79974.0,4383.0,330168.0,178974.0,151194.0,151194.0,194153.0,345346.0,261001.0,165500.0
867,SK이노베이션(주),96770,2012/12,16348024.0,563135.0,-34457.0,57238.0,354107.0,91903.0,262205.0,262205.0,-95218.0,166986.0,298278.0,162000.0
868,SK이노베이션(주),96770,2013/12,16915254.0,523321.0,60238.0,95254.0,-37056.0,25621.0,-62676.0,-62676.0,-91848.0,-154524.0,298278.0,122000.0
869,SK이노베이션(주),96770,2014/12,16061057.0,496172.0,37641.0,66174.0,-648157.0,-110505.0,-537652.0,-537653.0,152485.0,-385168.0,0.0,95400.0
870,SK이노베이션(주),96770,2015/12,17046336.0,730980.0,23042.0,341230.0,27239.0,86790.0,-59551.0,-189422.0,-125272.0,-314694.0,447386.0,172000.0
871,SK이노베이션(주),96770,2016/12,18305045.0,512926.0,72186.0,228568.0,351405.0,193892.0,157513.0,157513.0,231914.0,389428.0,596495.0,164500.0


# 딥러닝 모델 구축

# 잘 안되서 0 들어가있는 기타이익, 기타손실 뺴보고 해봄 - 이것도 잘 안됨

In [30]:
data = list_company3[1].loc[:,['순자산(백만원)', '판매비와 관리비(물류원가 등 포함)(백만원)',
       '법인세비용차감전순이익(백만원)', '법인세비용(백만원)', '계속영업이익(손실)(백만원)',
        '당기순이익(백만원)', '기타포괄손익(백만원)', '총포괄손익(백만원)', '배당금(백만원)', '3개월뒤종가(원)']]
data

,순자산(백만원),판매비와 관리비(물류원가 등 포함)(백만원),법인세비용차감전순이익(백만원),법인세비용(백만원),계속영업이익(손실)(백만원),당기순이익(백만원),기타포괄손익(백만원),총포괄손익(백만원),배당금(백만원),3개월뒤종가(원)
910,1557078.0,64996.0,65974.0,14586.0,51388.0,51388.0,24674.0,76062.0,55948.0,142000.0
911,1658352.0,46381.0,-52623.0,-25273.0,-27350.0,-27350.0,33294.0,5944.0,55948.0,105500.0
912,1559759.0,45121.0,-143257.0,-37981.0,-105276.0,-105276.0,-10873.0,-116149.0,41999.0,87800.0
913,1641582.0,42078.0,45095.0,12906.0,32190.0,32190.0,27687.0,59876.0,41999.0,78500.0
914,1663750.0,46572.0,9136.0,8973.0,163.0,163.0,-26518.0,-26355.0,22470.0,58200.0
915,1727901.0,54209.0,-26980.0,-2999.0,-23981.0,-23981.0,10573.0,-13408.0,22096.0,78500.0


In [31]:
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  # for reproducibility

x_data = data.loc[:,'순자산(백만원)':'배당금(백만원)']
x_data = x_data / 1000
y_data = pd.DataFrame(data.loc[:,'3개월뒤종가(원)'])


# Make sure the shape and data are OK
# print("x_data.shape : {}, \tlen(x_data) : {} \nx_data : \n{}".format(x_data.shape, len(x_data), x_data))
# print("-"*25)
# print("y_data.shape : {}  \ny_data : \n{}".format(y_data.shape, y_data))

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 9])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([9, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')


# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [32]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(2001):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})

    if step % 100 == 0 or step < 10 :
        print("\nStep : {} \nCost : {} \nPrediction :\n{}".format(step, cost_val, hy_val))


Step : 0 
Cost : 8889296896.0 
Prediction :
[[1119.0128]
 [1304.992 ]
 [1326.8345]
 [1217.1187]
 [1304.9053]
 [1399.9042]]

Step : 1 
Cost : 22345932603392.0 
Prediction :
[[4586728.5]
 [4887380. ]
 [4598154. ]
 [4832863.5]
 [4896372.5]
 [5088333. ]]

Step : 2 
Cost : 6.185909540932813e+16 
Prediction :
[[-2.3651462e+08]
 [-2.5220146e+08]
 [-2.3750091e+08]
 [-2.4926200e+08]
 [-2.5267712e+08]
 [-2.6259088e+08]]

Step : 3 
Cost : 1.7124721199187965e+20 
Prediction :
[[1.2449171e+10]
 [1.3274541e+10]
 [1.2500384e+10]
 [1.3120041e+10]
 [1.3299543e+10]
 [1.3821341e+10]]

Step : 4 
Cost : 4.740710544871851e+23 
Prediction :
[[-6.5500873e+11]
 [-6.9843563e+11]
 [-6.5770416e+11]
 [-6.9030642e+11]
 [-6.9975107e+11]
 [-7.2720541e+11]]

Step : 5 
Cost : 1.3123912459408246e+27 
Prediction :
[[3.4463315e+13]
 [3.6748227e+13]
 [3.4605139e+13]
 [3.6320508e+13]
 [3.6817441e+13]
 [3.8261951e+13]]

Step : 6 
Cost : 3.633149706839076e+30 
Prediction :
[[-1.8132889e+15]
 [-1.9335097e+15]
 [-1.8207509e+15

## 샤비어 함수

In [33]:
def xavier_init(n_inputs, n_outputs, uniform=True):
    if uniform:
        # 6 was used in the paper.
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        # 3 gives us approximately the same limints as above since this repicks
        # values greater than 2 standard deviations from the mean.
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

## 히든레이어 증가

In [34]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

tf.set_random_seed(777)  # for reproducibility

x_data = data.loc[:,'순자산(백만원)':'배당금(백만원)']
x_data = x_data / 1000
y_data = pd.DataFrame(data.loc[:,'3개월뒤종가(원)'])


# Make sure the shape and data are OK
# print("x_data.shape : {}, \tlen(x_data) : {} \nx_data : \n{}".format(x_data.shape, len(x_data), x_data))
# print("-"*25)
# print("y_data.shape : {}  \ny_data : \n{}".format(y_data.shape, y_data))

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 9])
Y = tf.placeholder(tf.float32, shape=[None, 1])


W1 = tf.get_variable("W1", shape=[9, 32], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([32]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)


W2 = tf.get_variable("W2", shape=[32, 64], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([64]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[64, 64], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([64]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)


W4 = tf.get_variable("W4", shape=[64, 64], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([64]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)


W5 = tf.get_variable("W5", shape=[64, 1], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([1]))
L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)

hypothesis = tf.matmul(L4, W5) + b5



# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [35]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(2001):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})

    if step % 100 == 0 or step < 10 :
        print("\nStep : {} \nCost : {} \nPrediction :\n{}".format(step, cost_val, hy_val))


Step : 0 
Cost : 9092549632.0 
Prediction :
[[126.845085]
 [145.26659 ]
 [159.76343 ]
 [137.37001 ]
 [158.6117  ]
 [158.63615 ]]

Step : 1 
Cost : 1.9722201131441747e+35 
Prediction :
[[4.2286303e+17]
 [4.5044854e+17]
 [4.2366416e+17]
 [4.4552733e+17]
 [4.5131826e+17]
 [4.6897854e+17]]

Step : 2 
Cost : nan 
Prediction :
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]

Step : 3 
Cost : nan 
Prediction :
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]

Step : 4 
Cost : nan 
Prediction :
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]

Step : 5 
Cost : nan 
Prediction :
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]

Step : 6 
Cost : nan 
Prediction :
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]

Step : 7 
Cost : nan 
Prediction :
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]

Step : 8 
Cost : nan 
Prediction :
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]

Step : 9 
Cost : nan 
Prediction :
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]

Step : 100 
Cost : nan 
Prediction :
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]